imports

In [1]:
from tqdm.notebook import tqdm as tqdm
import os
import json
import pandas as pd

from HGNN.train.configParser import ConfigParser, getModelName, getDatasetName
from myhelpers import TrialStatistics

parameters

In [2]:
experimentsPath="/raid/elhamod/Fish/experiments/"
dataPath="/raid/elhamod/Fish/"
experimentName="Fish30-5runs"

cuda=1

file and changes

In [3]:
old_params_file = os.path.join(experimentsPath, experimentName, "params.json")

# Keep same order as ConfigParserWriter!
changes = {
#     "grayscale": False, #Keep False
#     "tl_extralayer": False, #Keep False
#     "random_fitting": False, #Keep False
}
removes = ["noSpeciesBackprop", "phylogeny_loss_epsilon"]

prepare

In [4]:
config_parser = ConfigParser(experimentsPath, dataPath, experimentName)
paramsIterator = config_parser.getExperiments()  
number_of_experiments = sum(1 for e in paramsIterator)

# populate a hash table of all experiments, indexed by trial hash
experiments = {}
for i, experiment_params in enumerate(config_parser.getExperiments(fixExperiments=False)):
    num_of_models = experiment_params["numOfTrials"]
    for k in range(num_of_models):
        old_trialHash = TrialStatistics.getTrialName(experiment_params, k)
        experiment_params_new = {**experiment_params, **changes} 
        for key in removes:
            experiment_params_new.pop(key, None)
        experiment_params_new = config_parser.fixExperimentParams(experiment_params_new)
        new_trialHash = TrialStatistics.getTrialName(experiment_params_new, k)
        experiments[old_trialHash] = {"trialHash": new_trialHash}

Fix models

In [5]:
with tqdm(total=number_of_experiments, desc="experiment") as bar:
    for i, experiment_params in enumerate(config_parser.getExperiments(fixExperiments=False)):
        print(experiment_params)
        experiment_params_new = {**experiment_params, **changes} 
        for key in removes:
            experiment_params_new.pop(key, None)
        experiment_params_new = config_parser.fixExperimentParams(experiment_params_new)

        num_of_models = experiment_params["numOfTrials"]
        with tqdm(total=num_of_models, desc="trial") as bar2:
            for k in range(num_of_models):
            
                # Fix model names
                old_model_name = getModelName(experiment_params, k)
                old_modelPath = os.path.join(experimentsPath, experimentName, old_model_name)
                new_model_name = getModelName(experiment_params_new, k)
                new_modelPath = os.path.join(experimentsPath, experimentName, new_model_name)

                try:
                    if os.path.exists(old_modelPath):
                        os.rename(old_modelPath, new_modelPath)
                        print("model", old_modelPath, '->', new_modelPath, "updated")

                        j = json.dumps(experiment_params_new)
                        json_fileName = os.path.join(new_modelPath, 'params.json')
                        f = open(json_fileName,"w")        
                        f.write(j)
                        f.close()  
                        print("json", json_fileName, "updated")
                    else:
                        raise
                    
                except:
                    print("model", old_model_name, "could not be fixed to", new_model_name)
                    pass  
                
                

                bar2.update()
                
                experiments[TrialStatistics.getTrialName(experiment_params, k)]['modelName'] = new_model_name

        bar.update()

{'image_path': 'Curated4/Easy_30', 'suffix': None, 'img_res': 448, 'augmented': True, 'batchSize': 128, 'learning_rate': 0.001, 'numOfTrials': 5, 'fc_layers': 1, 'pretrained': True, 'epochs': 120, 'patience': -1, 'optimizer': 'adabelief', 'scheduler': 'plateau', 'weightdecay': 0.01, 'scheduler_gamma': 0.5, 'scheduler_patience': 10, 'modelType': 'BB', 'lambda': 0, 'two_phase_lambda': False, 'tl_model': 'ResNet18', 'link_layer': 'avgpool', 'adaptive_smoothing': True, 'adaptive_lambda': 0.1, 'adaptive_alpha': 0.5, 'tripletEnabled': False, 'regularTripletLoss': False, 'tripletSamples': 2, 'tripletSelector': 'semihard', 'tripletMargin': 2, 'triplet_layers_dic': 'layer2,layer4', 'L1reg': False, 'phylogeny_loss': False, 'displayName': 'Fish30-5runs-BB', 'grayscale': False, 'tl_extralayer': False, 'random_fitting': False}


model models/b0bbf384d840248a268600aace11b814aede8a170b9587f7b0355995 could not be fixed to models/b5cfd22bc25db4d6d4b78458ecf25d3654a5a1b90f31b7644c774dfd
model models/d2da6db6e2611ab9bbf71afea950f95f5ce1ceed59ca8911b81123c6 could not be fixed to models/6248e2990a7fa2dcf904bae0708297b7289d2309954ffb6ddd9e92ef
model models/bdcbbf2053975675fdf24c1ae6ee5aaa745383b05f90ef3bdc1c36d4 could not be fixed to models/c36cbc879e882fc5816225f7a6321eca2711e2183dddd582b4ed6c7c
model models/39f70f731446a9fe281f68e7e1f07d9a557776480cc9f5c06dd9e48a could not be fixed to models/46dcfe815086eb73fd1cde88631d287a805bc369fd9682ec7b039e94
model models/7190f5b9f51079f88b52845e6664ad12124fc7520d6493391a42d4a8 could not be fixed to models/62f98545fd794cb603a9396235f933dec18a595d8a48c63857c56b5d

{'image_path': 'Curated4/Easy_30', 'suffix': None, 'img_res': 448, 'augmented': True, 'batchSize': 128, 'learning_rate': 0.001, 'numOfTrials': 5, 'fc_layers': 1, 'pretrained': True, 'epochs': 120, 'patience': -1, 'optim

model models/995148581b09dd2ad1ea63604f3184e46b6910299e61f7b812c9c37f could not be fixed to models/18e6c6725224e23845f06e0e3d3152228fd37b5180f26b0b807804f3
model models/237bf8cf570d3f19dfcf78ca129e28b5456e43dd5a95e0c7387f124e could not be fixed to models/920ddacbd41b6d83c16fcc7868e744b70327a68b9fb446a7caea6b2e
model models/8123c396d8c27bb0b6b8f1b2e2aaa43958dc0fa0878fca114f413fc5 could not be fixed to models/6ce2f4f14a1600de25431c33232a60e9062e29aee6b984e19ec292ce
model models/c54fdf8e69327fb22d82977ba95c60784d46f5c8ea7dcea66874c0f1 could not be fixed to models/734ab74b41c17fe0c645813fddf4a4a670802f93e9a38951defd1dd8
model models/e6dd82536bf4c3f35a0e711d0923369e27d111f72b4fc3a248541b25 could not be fixed to models/c5bcddf034dadf2253419e8925b2627dfdf15c906d61a5606a8fd484

{'image_path': 'Curated4/Easy_30', 'suffix': None, 'img_res': 448, 'augmented': True, 'batchSize': 100, 'learning_rate': 0.001, 'numOfTrials': 5, 'fc_layers': 1, 'pretrained': True, 'epochs': 120, 'patience': -1, 'optim

model models/9eb8fea5187552fd8ba89d7b6dc4e838dfafb2acad10467cc11f54d7 could not be fixed to models/7660211da44ec5c0c2f97b58ccce8a7ee7ee57e93d8fbb225c9c3c4a
model models/35682b2817619f2423914bc320437e010b4278eeaa3fde103c7def06 could not be fixed to models/9d49fd9831f6bfa381d77e5772b0f3c237260949eb43961eda571115
model models/67c27db3ea454640001e86a71ecb8b29938c7bbfd3e60f07b2786733 could not be fixed to models/ca6ada5fad2594539f51ad3619a1aaf87d5bb7cf02c36607df465a48
model models/fb9f11364acd776759f3da37a54b35aa1073b49177b4fd51cf426593 could not be fixed to models/bb98f1dbb0465b21c88a98484b7eb62860f360a3100e20719779bb91
model models/1142a0789e8b98d7d025f68ecd14fe5d802e760067812c266c740e48 could not be fixed to models/3370cf706f9fc004b184097e4782b01579e4ef552dbc4f6c87d48333

{'image_path': 'Curated4/Easy_30', 'suffix': None, 'img_res': 448, 'augmented': True, 'batchSize': 100, 'learning_rate': 0.001, 'numOfTrials': 5, 'fc_layers': 1, 'pretrained': True, 'epochs': 120, 'patience': -1, 'optim

model models/0656417f7a4b50b369895ff569d543705125536bba39f4374f211273 could not be fixed to models/65bfd4e272bad9f0b499448c6c26aa3f9ca81bb83157d485f531cfea
model models/21f6d364c39fda6a926e846150e0113ec838cffcb8af59054070ce6e could not be fixed to models/861df63a7b35bf1076de7144792edca377eb9fcb73569a18f2d44e9f
model models/a8af002c123ff76fefdbfa6605b791be19a530da0575144cef368172 could not be fixed to models/b0baabb26eda91fc4faeea5315708b881dc1dd052002c341dc8efc3d
model models/9ccc2dc836ae1e8d53668422ea076af542d73d57eb1bc63a4e3246f8 could not be fixed to models/419220d370d691a3e6a23c9119f60860fae3b20933c60f3b5d056cce
model models/489c0fc7fd1bc1e79125a0965c747549dae78ce09cd18e7161df9473 could not be fixed to models/3d3454e8256bc182a3919860cec4e2045ee577cc764aad04953a16e5

{'image_path': 'Curated4/Easy_30', 'suffix': None, 'img_res': 448, 'augmented': True, 'batchSize': 100, 'learning_rate': 0.001, 'numOfTrials': 5, 'fc_layers': 1, 'pretrained': True, 'epochs': 120, 'patience': -1, 'optim

model models/9678fd33e9d68c7452695259762844be2d0ea2fab2ac2888d74c0ce4 could not be fixed to models/0699a77ccde53a439c5b79438c9662bc6b8828cb0acb9cbf4f46946b
model models/3cea378b1ce62143d0ab1291eda6784552b9cf3f23c8daabbfb5687b could not be fixed to models/e0e437d3cd9138a00cc0f8a120f838c1232ab468aa04ab62fdeecfbd
model models/e76a0a1800beaf471452834287c8c3bf82abca143ba42633fe02ba75 could not be fixed to models/43d8b2269aafa18675755722f7401c4dfd3ec047994105029fc4b093
model models/0d72b1295b7193592a3e5c6c303f06d453061531c6c37fa562f4f974 could not be fixed to models/b5cc3129fa4d0f29dd3996aed29f2c51479fa36059c5377b31ab0a1b
model models/6394e15ba8acd486548087eff24690484119be9985fcd4bdb9c69968 could not be fixed to models/5dc68fc9ff9ef490d16fb43e69c42ff631c7805743e9c051b61fed7b




Fix results directory

In [6]:
with tqdm(total=number_of_experiments, desc="experiment") as bar:
    for i, experiment_params in enumerate(config_parser.getExperiments(fixExperiments=False)):
        experiment_params_new = {**experiment_params, **changes} 
        for key in removes:
            experiment_params_new.pop(key, None)
        experiment_params_new = config_parser.fixExperimentParams(experiment_params_new)

        old_experimentHash = TrialStatistics.getTrialName(experiment_params)
        old_experimentPath = os.path.join(experimentsPath, experimentName, "results", old_experimentHash)
        new_experimentHash = TrialStatistics.getTrialName(experiment_params_new)
        new_experimentPath = os.path.join(experimentsPath, experimentName, "results", new_experimentHash)
        
        try:
            if os.path.exists(old_experimentPath):

                os.rename(old_experimentPath, new_experimentPath)
                print("result", old_experimentPath, '->', new_experimentPath, "updated")

                j = json.dumps(experiment_params_new)
                json_fileName = os.path.join(new_experimentPath, 'params.json')
                f = open(json_fileName,"w")        
                f.write(j)
                f.close()  
                print("json", json_fileName, "updated")
            else:
                raise
                
        except:
            print("result", old_experimentHash, "could not be fixed to",new_experimentHash)
            pass 
        
        experiments[TrialStatistics.getTrialName(experiment_params, 0)]['experimentHash'] = new_experimentHash
        
        bar.update()

result f86cf27555637bc5cf8a5f760d9617e2a21d58bcd63de2fc12029d09 could not be fixed to 2e215a7fb9e214f45d0bcbd88a008c5cf6e1d534cf592fb95d272046
result 7f6c7aebc719d403722383b3af0dcec16db4143a843913d96e85e7b0 could not be fixed to fb86b38bc14f5096064fcc5c3ffb798e6034a0475a14b08f499c1ecf
result e2081d8eb0ce26b4d19970ea5d1beb8ef22e8d1876ceafad12aca00f could not be fixed to a2f43a4fa72c1e15c319f4d78891f28cb8d872b0c757fbaed4d6bf42
result cc73809577ce246c1e3d33d5fd70fcc7b820b69ca193863ed6106fb6 could not be fixed to a43d4b6a7c0502766a0b03e47168418c71b02c9c5af38740fd058421
result 9bef26f13bc9e5bba63902eff80f862941d91a02fb52e42c2042cf62 could not be fixed to 57aa6370a8c4fe21b40ce550fbb3cde94d90b50aebdd50cc0f676aed



Fix experiments file

In [7]:
# open experiments.csv file
experimentsFileNameAndPath=os.path.join(experimentsPath, "experiments.csv")
experiments_file= pd.read_csv(experimentsFileNameAndPath)

# For all experiments
with tqdm(total=number_of_experiments, desc="experiment") as bar:
    for i, experiment_params in enumerate(config_parser.getExperiments(fixExperiments=False)):
        
        numOfTrials = experiment_params["numOfTrials"]
        trialHash_0 = TrialStatistics.getTrialName(experiment_params, 0) # used for keys who don't change for different trials
        for key in removes:
            experiment_params_new.pop(key, None)
        experiment_params_new = {**experiment_params, **changes} 
        experiment_params_new = config_parser.fixExperimentParams(experiment_params_new)
        
        # For all trials
        with tqdm(total=num_of_models, desc="trial") as bar2:
            for j in range(numOfTrials):
                
                trialHash = TrialStatistics.getTrialName(experiment_params, j)
                try:
                    # If old entry doesn't exist, nothing to update
                    record_exists = not (experiments_file[experiments_file['trialHash'] == trialHash][experiments_file['experimentName'] == experimentName]).empty
                    if record_exists:
                        
                        # drop old row and caluclate new row
                        new_trialHash = experiments[trialHash]["trialHash"]
                        experiments_file.drop(experiments_file[experiments_file['trialHash'] == trialHash][experiments_file['experimentName'] == experimentName].index, inplace = True) 
                        row_information = {
                            'experimentName': experimentName,
                            'modelName': experiments[trialHash]["modelName"],
                            'datasetName': experiments[trialHash_0]["datasetName"],
                            'experimentHash': experiments[trialHash_0]["experimentHash"],
                            'trialHash': new_trialHash
                        }
                        row_information = {**row_information, **experiment_params_new} 

                        # error handling to avoid duplicates
                        record_exists = not (experiments_file[experiments_file['trialHash'] == new_trialHash][experiments_file['experimentName'] == experimentName]).empty
                        if record_exists:
                            experiments_file.drop(experiments_file[experiments_file['trialHash'] == new_trialHash][experiments_file['experimentName'] == experimentName].index, inplace = True) 

                        # update with new entry    
                        experiments_file = experiments_file.append(pd.DataFrame(row_information, index=[0]), ignore_index = True)
                        print("trialHash", trialHash, "->", new_trialHash, "updated")
                except:
                    print("trialHash", trialHash, "could not be updated!")
                    pass
                bar2.update()
        bar.update()

# resave file
experiments_file.to_csv(experimentsFileNameAndPath, header=True, index=False)

/home/elhamod/melhamodenv3/lib/python3.6/site-packages/ipykernel_launcher.py:23: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


Fix params file

In [8]:
experiment_list = []
with tqdm(total=number_of_experiments, desc="experiment") as bar:
    for i, experiment_params in enumerate(config_parser.getExperiments(fixExperiments=False)):
        
        experiment_params_new = {**experiment_params, **changes} 
        for key in removes:
            experiment_params_new.pop(key, None)
        print(experiment_params_new)
        experiment_list.append(experiment_params_new)
        bar.update()


j = json.dumps({"experimentList": experiment_list})
f = open(old_params_file,"w")        
f.write(j)
f.close()  
print("json", old_params_file, "updated")

{'image_path': 'Curated4/Easy_30', 'suffix': None, 'img_res': 448, 'augmented': True, 'batchSize': 128, 'learning_rate': 0.001, 'numOfTrials': 5, 'fc_layers': 1, 'pretrained': True, 'epochs': 120, 'patience': -1, 'optimizer': 'adabelief', 'scheduler': 'plateau', 'weightdecay': 0.01, 'scheduler_gamma': 0.5, 'scheduler_patience': 10, 'modelType': 'BB', 'lambda': 0, 'two_phase_lambda': False, 'tl_model': 'ResNet18', 'link_layer': 'avgpool', 'adaptive_smoothing': True, 'adaptive_lambda': 0.1, 'adaptive_alpha': 0.5, 'tripletEnabled': False, 'regularTripletLoss': False, 'tripletSamples': 2, 'tripletSelector': 'semihard', 'tripletMargin': 2, 'triplet_layers_dic': 'layer2,layer4', 'L1reg': False, 'phylogeny_loss': False, 'displayName': 'Fish30-5runs-BB', 'grayscale': False, 'tl_extralayer': False, 'random_fitting': False}
{'image_path': 'Curated4/Easy_30', 'suffix': None, 'img_res': 448, 'augmented': True, 'batchSize': 128, 'learning_rate': 0.001, 'numOfTrials': 5, 'fc_layers': 1, 'pretrained'